In [ ]:
import ee
from utils import get_pa_filter, set_geometry_type, get_biome

ee.Authenticate()
ee.Initialize(project='dse-staff')

PROTECTED_AREAS = ee.FeatureCollection("WCMC/WDPA/202106/polygons")

In [ ]:
filtered_polygon = PROTECTED_AREAS.map(set_geometry_type) \
    .filter(get_pa_filter("Polygon")) \
    .map(get_biome) 

wdpa_pids_ee = filtered_polygon.aggregate_array('WDPA_PID')
wdpaids = wdpa_pids_ee.getInfo()
len(wdpaids)  # 6358

In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=filtered_polygon,
    description='WDPA_polygons',
    fileFormat='GeoJSON'
)
task.start()

# After it finishes, download WDPA_polygons.geojson from your Google Drive to this repository's ..data/ folder
# Then proceed to the next step